In [2]:
import pandas as pd

## Some basic exploration of the CMU data original dataset
First, we opened the **CMU character metadata** and look at the missing values :

In [2]:
columns_character = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date', 'Character name', 'Actor date of birth',
                     'Actor gender', 'Actor height (in meters)', 'Actor ethnicity (Freebase ID)', 'Actor name',
                     'Actor age at movie release', 'Freebase character/actor map ID', 'Freebase character ID',
                     'Freebase actor ID']

df_cmu_character = pd.read_csv("MovieSummaries/character.metadata.tsv",sep='\t',names=columns_character)

print(df_cmu_character.shape)
print('Percentage of NaN in each feature : ')
print(df_cmu_character.isna().sum(axis = 0) / df_cmu_character.shape[0] * 100)

(450669, 13)
Percentage of NaN in each feature : 
Wikipedia movie ID                  0.000000
Freebase movie ID                   0.000000
Movie release date                  2.217814
Character name                     57.220488
Actor date of birth                23.552763
Actor gender                       10.120288
Actor height (in meters)           65.645740
Actor ethnicity (Freebase ID)      76.466542
Actor name                          0.272484
Actor age at movie release         35.084064
Freebase character/actor map ID     0.000000
Freebase character ID              57.218269
Freebase actor ID                   0.180842
dtype: float64


As we were first interested in looking at the ethnicities of the actors, we map the freebase ID with its label thanks to the `mid2name.tsv` file found on : https://github.com/xiaoling/figer/issues/6

In [3]:
df_mapID = pd.read_csv("Expanded_data/mid2name.tsv", sep='\t', names=['ID', 'label'])
df_mapID = df_mapID.drop_duplicates(subset=["ID"], keep='first')
ethnicity = df_cmu_character['Actor ethnicity (Freebase ID)']
df_ethnicity = ethnicity.to_frame()
df_ethnicity.columns = ['ID']
df_merge = pd.merge(df_ethnicity, df_mapID, how='left')
df_cmu_character['Ethnicity'] = df_merge['label']

We found that it is was difficult to complete the ethnicities of the actors with external datatset. So, we decided to choose another idea.

Then, we opened the **CMU movies metadata** and look at the missing values :

In [4]:
columns_movie = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue',
                 'Movie runtime', 'Movie languages (Freebase ID:name tuples)', 'Movie countries (Freebase ID:name tuples)',
                 'Movie genres (Freebase ID:name tuples)']
df_cmu_movie = pd.read_csv("MovieSummaries/movie.metadata.tsv",sep='\t', names=columns_movie)

print(df_cmu_movie.shape)
print('Percentage of NaN in each feature : ')
print(df_cmu_movie.isna().sum(axis = 0) / df_cmu_movie.shape[0] * 100)

print('\nSum of {} in the string columns : ')
print('Movie languages : {}'.format(sum(df_cmu_movie['Movie languages (Freebase ID:name tuples)']=='{}')))
print('Movie countries : {}'.format(sum(df_cmu_movie['Movie countries (Freebase ID:name tuples)']=='{}')))
print('Movie genres : {}'.format(sum(df_cmu_movie['Movie genres (Freebase ID:name tuples)']=='{}')))

(81741, 9)
Percentage of NaN in each feature : 
Wikipedia movie ID                            0.000000
Freebase movie ID                             0.000000
Movie name                                    0.000000
Movie release date                            8.443743
Movie box office revenue                     89.722416
Movie runtime                                25.018045
Movie languages (Freebase ID:name tuples)     0.000000
Movie countries (Freebase ID:name tuples)     0.000000
Movie genres (Freebase ID:name tuples)        0.000000
dtype: float64

Sum of {} in the string columns : 
Movie languages : 13866
Movie countries : 8154
Movie genres : 2294


Then, we looked at the CMU plot summaries data :

In [5]:
df_cmu_summaries = pd.read_csv("MovieSummaries/plot_summaries.txt",sep='\t', names=['Wikipedia movie ID', 'Plot summary'])
df_cmu_summaries.head(3)

print('Percentage of missing summaries : ')
print(100 - df_cmu_summaries.shape[0] / df_cmu_movie.shape[0] * 100)

Percentage of missing summaries : 
48.24751348772342


## Pulling data to complete our dataset

The CMU movie metadata contains not many and not recent movies (until 2012 only). Moreover, it has a lot of NA values, espcially for the box office revenue. So, we decided to complete this dataset to have more representative one. We use :
* **Wikipedia** to query box office revenues that were missing
* **IMDB** dataset to complete the amount of movies and ensure good representation of their variety
* **TMDB** dataset to fetch movie budget and country of origin (production)

If you want to run the notebooks, you have to download IMDB data from https://datasets.imdbws.com/. Please download these files :
* `title.akas.tsv.gz`
* `title.basics.tsv.gz`
* `title.ratings.tsv.gz`
Unzip them, and place them in `IMDB_data/`

The other data is obtained by running .py files, which make direct query using APIs.

### Wikipedia query
By running `wikipedia_query.py`, a `Expanded_data/wikipedia_query.tsv` file will be created. The script requests imdb-associated films on wikipedia, with associated box office revenues and freebase IDs if available (on wikipedia). With the freebase IDs we will be able to associate this data with the CMU movie metadata. You do not need to run this command as the `wikipedia_query.tsv` file was small enough to be pushed on Github.

In [6]:
#!python3 wikipedia_query.py

### Merge IMDB and Wikipedia data
By running `expand_data.py`, a `Expanded_data/big_data.tsv`file will ber created. The script brings together IMDB data with associated wikipedia data and notably box office values from the CMU dataset to create a big representative movie dataset, used for large-scale analysis. So, we have :
* `Expanded_data/big_data.tsv` which is the merge between IMDB, wikipedia query data, and info from our original dataset.

In [5]:
#!python3 expand_data.py

### TMDB query
By running `TMDB_query.py`, a `Expanded_data/TMDB_query.tsv` file will be created. The script uses the imdb ids and freebase ids to query movie budgets and country of origin (production).

In [9]:
#!python3 TMDB_query.py

### Merge TMDB to big_data
By running `expand_data_2.py`, `Expanded_data/big_data.tsv` will be updated. The script adds budget and revenue to our previous big data using the index corresponding to box office revenue.
TO DO: ADD INFLATION AND OTHER USEFUL INFORMATION, AS WELL AS BIG DATASET PREPROCESSING THROUGH THIS SCRIPT, SUCH THAT THE BIG DATASET IS FINISHED. THEN EXPORT TO CMU DATASET ?

In [ ]:
# !python3 expand_data_2.py

TEMPORARY STATE OF big_data:

In [3]:
big_data = pd.read_csv('Expanded_data/big_data.tsv', sep='\t')

C:\Users\loris\AppData\Local\Temp\ipykernel_6852\4229692301.py:1: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  big_data = pd.read_csv('Expanded_data/big_data.tsv', sep='\t')


In [4]:
big_data.head(2)

,Freebase movie ID,IMDB_id,Movie box office revenue,Movie genres names,Movie name,Movie release date,averageRating,budget,numVotes,prod_country
0,/m/0100_m55,tt0138297,NaN,"['Comedy', 'Sci-Fi']",Urban Animals,1987,5.2,NaN,79.0,NaN
1,/m/0100_mnm,tt0202813,NaN,['Comedy'],NaN,1999,5.8,NaN,15.0,NaN


In [ ]:
big_data.count()

### Inflation
As we are interested in the analyses of the box office revenues, we have to take care of the inflation. So, we found data on the inflation of each country from 1960 to 2021 on https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG

In [9]:
# Inflation per country from 1960 to 2021
col = ["Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020", "2021", "2022"]
df_inf = pd.read_csv("Inflation_data/inflation.csv", names=col, sep=',', skiprows=[0,1,2,3,4])

# Delete 2022 column because all are NaN
df_inf = df_inf.iloc[:,0:-1]

# Metadata to link the country code with some information about the Region, Income and other stuffs.
df_metadata_inf = pd.read_csv("Inflation_data/inflation_metadata.csv", sep=',')

We extracted from this dataset, the information that are useful for our analyzes :

In [11]:
# First the world inflation mean
df_world_inflation = df_inf.iloc[:,3:df_inf.shape[1]]
world_inf_mean = df_world_inflation.mean(axis=0)

# Then, the inflation specifically in the USA
us_inf = df_inf[df_inf['Country Code'] == 'USA']

# Finally, the inflation in the high income countries only
df_meta_HI = df_metadata_inf[df_metadata_inf['IncomeGroup'] == 'High income']
df_HI_inf = df_inf[df_inf['Country Code'].isin(df_meta_HI['Country Code'])]
HI_inf_mean = df_HI_inf.iloc[:,3:df_HI_inf.shape[1]].mean(axis=0)

## Preprocessing
DO THE WHOLE CMU COMPLETION USING THE BIG DATASET AND PREPROCESSING IN A .PY ?

In [12]:
from preprocessing import *

First, we load the `Expanded_data/movie.expanded_metadata.tsv` file :

In [13]:
df_movie = pd.read_csv("Expanded_data/movie.expanded_metadata.tsv",sep='\t')
print(df_movie.shape)
df_movie.head(3)

(81741, 9)


,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples)
0,142780.0,/m/011_mj,The Circus,1928-01-06,NaN,68.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/06ppq"": ""S..."
1,142786.0,/m/011_p6,Thunderbolt,1929,NaN,91.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/01g6gs"": ""Black-an..."
2,142822.0,/m/011_zy,The Green Goddess,1930-02-13,NaN,73.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02rd8h3"": ""Goat gland"", ""/m/07s9rl0"": ""Dr..."


The `Expanded_data/movie.expanded_metadata.tsv` is now our CMU movies metadata whose missing values were completed as much as possible. We noticed that the Movie language, Movie countries and Movie genres were of type string and it was not easy to work with that. So, we decided, to change these columns into a dictionnary type and to create new columns with only the list of the names. We do this preprocessing with functions that are in `preprocessing.py`.

In [14]:
# Change the (Freebase ID:name tuples) columns from strings to dictionnaries
df_movie['Movie genres (Freebase ID:name tuples)'] = df_movie['Movie genres (Freebase ID:name tuples)'].apply(lambda x: transform_into_dict(x))
df_movie['Movie countries (Freebase ID:name tuples)'] = df_movie['Movie countries (Freebase ID:name tuples)'].apply(lambda x: transform_into_dict(x))
df_movie['Movie languages (Freebase ID:name tuples)'] = df_movie['Movie languages (Freebase ID:name tuples)'].apply(lambda x: transform_into_dict_2(x))

# New column with the list of names only --> easier to do analyses
df_movie['Movie languages names'] = df_movie['Movie languages (Freebase ID:name tuples)'].apply(lambda x: transform_to_list_names(x))
df_movie['Movie countries names'] = df_movie['Movie countries (Freebase ID:name tuples)'].apply(lambda x: transform_to_list_names(x))
df_movie['Movie genres names'] = df_movie['Movie genres (Freebase ID:name tuples)'].apply(lambda x: transform_to_list_names(x))

# As the Movie release date is not homogeneous across all movies, we decided to only keep the year as a timestamp.
df_movie['Movie release date'] = pd.to_datetime(df_movie['Movie release date'], errors='coerce').dt.year.astype('Int64')

df_movie.head(3)

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples),Movie languages names,Movie countries names,Movie genres names
0,142780.0,/m/011_mj,The Circus,1928,NaN,68.0,"{'/m/06ppq': 'Silent film', 'm/02h40lc': 'Engl...",{'/m/09c7w0': 'United States of America'},"{'/m/06cvj': 'Romantic comedy', '/m/06ppq': 'S...","[Silent film, English Language]",[United States of America],"[Romantic comedy, Silent film, Adventure, Blac..."
1,142786.0,/m/011_p6,Thunderbolt,1929,NaN,91.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/07s9rl0': 'Drama', '/m/01g6gs': 'Black-an...",[English Language],[United States of America],"[Drama, Black-and-white]"
2,142822.0,/m/011_zy,The Green Goddess,1930,NaN,73.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02rd8h3': 'Goat gland', '/m/07s9rl0': 'Dr...",[English Language],[United States of America],"[Goat gland, Drama, Indie, Black-and-white]"


Then, we can also merge the plot summaries to the df_movie :

In [15]:
# Merge with the df_movie thanks to the wikipedia movie ID
df_movie = pd.merge(df_movie, df_cmu_summaries, how='left')

Then, we can load the bigger dataset `Expanded_data/IMDB_wiki.tsv` :

In [22]:
df_movie_big = pd.read_csv("Expanded_data/IMDB_wiki.tsv",sep='\t')
print(df_movie_big.shape)
df_movie_big.head(3)

C:\Users\loris\AppData\Local\Temp\ipykernel_9128\1971059664.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie_big = pd.read_csv("Expanded_data/IMDB_wiki.tsv",sep='\t')


(9343301, 8)


,IMDB_id,Freebase movie ID,Movie name,Movie release date,Movie genres names,averageRating,numVotes,Movie box office revenue
0,tt0000001,NaN,Carmencita,1894.0,"['Documentary', 'Short']",5.7,1921.0,NaN
1,tt0000002,NaN,The Clown and His Dogs,1892.0,"['Animation', 'Short']",5.8,260.0,NaN
2,tt0000003,NaN,NaN,1892.0,"['Animation', 'Comedy', 'Romance']",6.5,1729.0,NaN


As for the CMU dataset df_movie, we only keep the Movie release year.
DO THIS PREPROCESSING IN LAST BIG DATA SCRIPT ?

In [23]:
df_movie_big['Movie release date'] = pd.to_datetime(df_movie_big['Movie release date'], errors='coerce').dt.year.astype('Int64')

We noticed that there are either 'NaN' or '\\N' for missing values in this dataset. So, we changed the '\\N' into 'NaN' for more consistency across the dataset.

In [24]:
#print('\nSum of in the string columns : ')
#print((df_movie_big == '\\N').sum(axis = 0) / df_movie_big.shape[0] * 100)

# float('NaN') otherwise it is not recognized as the other NaN of the dataset
df_movie_big.replace('\\N', float('NaN'), inplace=True)

print('Percentage of NaN in each feature : ')
print(df_movie_big.isna().sum(axis = 0) / df_movie_big.shape[0] * 100)

Percentage of NaN in each feature : 
IMDB_id                      0.000000
Freebase movie ID           98.902476
Movie name                  86.257962
Movie release date          13.435808
Movie genres names           0.036796
averageRating               86.695955
numVotes                    86.695955
Movie box office revenue    99.967324
dtype: float64


As we are interested into the box office revenue, we look at this dataset when dropping the missing box office revenues :

In [21]:
df_movie_box = df_movie_final.dropna(subset='Movie box office revenue')
print(df_movie_box.shape)
df_movie_box.head(3)

(10210, 7)


,Freebase movie ID,Movie name,Movie release date,Movie genres names,averageRating,numVotes,Movie box office revenue
9,/m/0100blym,Testament of Youth,2014,"['Biography', 'Drama', 'History']",7.2,29135.0,1800000.0
33,/m/0100khzv,Leviathan,2014,"['Crime', 'Drama']",7.6,53818.0,4100000.0
231,/m/0105j_71,Spy,2015,"['Action', 'Comedy']",7.0,245758.0,235700000.0


We have 10'200 movies to do our analyses on the box office revenenues. We suppose that this sample is large enough to get reliable results. However, we have to verify this hypothese by doing some further analyses